In [1]:
## Necessary packages
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import warnings
warnings.filterwarnings("ignore")

import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from torch import optim
from itertools import chain
from utils import batch_generator
import torch.nn.functional as F
from sklearn.metrics import precision_score, recall_score, roc_auc_score, f1_score, mean_squared_error, accuracy_score

In [2]:
def MinMaxScaler(data):
    """Min Max normalizer.

    Args:
    - data: original data

    Returns:
    - norm_data: normalized data
    """
    numerator = data - np.min(data, 0)
    denominator = np.max(data, 0) - np.min(data, 0)
    norm_data = numerator / (denominator + 1e-7)
    return norm_data

def StandardizationScaler(data):
    mean, std = np.mean(data, 0), np.mean(data, 0)
    norm_data = (data - mean)/(std + 1e-7)
    return norm_data

  
def real_data_loading(absolute_path):
    """Load and preprocess real-world datasets.
  
    Args:
    - absolute_path: absoulte_path
    - seq_len: sequence length

    Returns:
    - data: preprocessed data.
    """  
    ori_data = pd.read_csv(absolute_path, index_col= 0)
    cols = ["subset"]
    ori_data = ori_data.drop(columns = cols)
    ori_data = ori_data.values
    # First dimension is time or last dimension is label #ori_data = ori_data[:, :-1]
    ori_data = ori_data.astype(dtype = np.float64)
    # Normalize the data
    ori_data = MinMaxScaler(ori_data)
    return np.array(ori_data)



def compute_measures(y_tilde, y):
    y_pred = np.round(y_tilde.to('cpu').detach(), decimals = 0)
    y = np.round(y.to('cpu').detach(), decimals = 0)
    acc = accuracy_score(y, y_pred)
    prec = precision_score(y, y_pred)
    recall = recall_score(y, y_pred)
    f1 = f1_score(y, y_pred)
    return acc, prec, recall, f1

In [3]:
#set the device 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Data loading
data_path = "data/"
path_train_data = data_path + 'train_data.csv'
path_valid_data = data_path + 'valid_data.csv'
path_test_data = data_path + 'test_data.csv'

#preprocessing the data.
train_data = real_data_loading(path_train_data)   
valid_data = real_data_loading(path_valid_data)
test_data = real_data_loading(path_test_data)
disc = torch.load("pre_trained_model/discriminator.mdl")
disc

Discriminator(
  (fc1): Linear(in_features=196, out_features=96, bias=True)
  (fc2): Linear(in_features=96, out_features=96, bias=True)
  (fc3): Linear(in_features=96, out_features=1, bias=True)
  (leaky_relu): LeakyReLU(negative_slope=0.2)
)

In [4]:
model = nn.ModuleList([])
for index, layer in enumerate(disc.children()):
    if index==1:
        model.append(nn.LeakyReLU(0.2))
        model.append(nn.Dropout(0.2))
        model.append(nn.BatchNorm1d(96))
    else: 
        if not isinstance(layer, torch.nn.LeakyReLU):
            model.append(layer)
model.append(nn.Sigmoid())

ModuleList(
  (0): Linear(in_features=196, out_features=96, bias=True)
  (1): LeakyReLU(negative_slope=0.2)
  (2): Dropout(p=0.2, inplace=False)
  (3): BatchNorm1d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (4): Linear(in_features=96, out_features=1, bias=True)
  (5): Sigmoid()
)

In [5]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.fc1 = nn.Linear(in_features=196, out_features=96, bias=False)
        self.leaky_relu = nn.LeakyReLU(0.2)
        self.dropout = nn.Dropout(0.2)
        self.fc3 = nn.Linear(in_features=96, out_features=1)
        self.sigmoid = nn.Sigmoid()
        self.bn = nn.BatchNorm1d(96)
        nn.init.xavier_uniform_(self.fc1.weight)
        nn.init.xavier_uniform_(self.fc3.weight)
        
    def forward(self, x):
        x = self.fc1(x)
        x = self.leaky_relu(x)
        x = self.dropout(x)
        x = self.bn(x)
        x = self.fc3(x)
        x = self.sigmoid(x)
        return x

In [6]:
model = nn.Sequential(*model).to(device)
model
"""
model = Model()
model = model.to(device)
model
"""

'\nmodel = Model()\nmodel = model.to(device)\nmodel\n'

In [7]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_data, batch_size=8, shuffle=True)
valid_dataloader = DataLoader(valid_data, batch_size=8, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=8, shuffle=False)

In [8]:
optimizer = optim.Adam(model.parameters(), lr= 0.00001)
criterion = nn.BCELoss()


for epoch in range(200):
    model.train()
    train_loss = 0
    count = 0
    train_acc = 0
    train_f1 = 0
    for item in train_dataloader:
        x, y = item[:, :-1], item[:, -1].reshape(-1, 1)
        y = y.double().to(device)
        x = x.float().to(device)
        optimizer.zero_grad()
        y_tilde = model(x)
        y_tilde = y_tilde.double().to(device)
        loss  = criterion(y_tilde, y)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        
        acc, prec, recall, f1 = compute_measures(y_tilde, y)
        train_acc += acc
        train_f1+=f1
        
    model.eval()
    valid_loss = 0
    count = 0
    valid_acc = 0
    valid_f1 = 0
    for item in valid_dataloader:
        with torch.no_grad():
            x, y = item[:, :-1], item[:, -1].reshape(-1, 1)
            y = y.double().to(device)
            x = x.float().to(device)
            y_tilde = model(x)
            y_tilde = y_tilde.double().to(device)
            loss  = criterion(y_tilde, y)
            valid_loss += loss.item()
            
            acc, prec, recall, f1 = compute_measures(y_tilde, y)
            valid_acc += acc
            valid_f1+=f1
        
    print("Epoch {}; train_loss: {}; val_loss: {}; train_acc: {}; train_f1: {}, val_acc: {}; val_f1: {}".format(epoch, 
                                                                                  np.round(train_loss/len(train_dataloader), decimals=3), 
                                                                                  np.round(valid_loss/len(valid_dataloader), decimals=3), 
                                                                                  np.round(train_acc/len(train_dataloader), decimals=3), 
                                                                                  np.round(train_f1/len(train_dataloader), decimals=3),
                                                                                  np.round(valid_acc/len(valid_dataloader), decimals=3),
                                                                                  np.round(valid_f1/len(valid_dataloader), decimals=3)
                                                                                  ))
                                        
    
    

Epoch 0; train_loss: 0.745; val_loss: 1.05; train_acc: 0.518; train_f1: 0.549, val_acc: 0.6; val_f1: 0.692
Epoch 1; train_loss: 0.736; val_loss: 1.009; train_acc: 0.515; train_f1: 0.543, val_acc: 0.608; val_f1: 0.733
Epoch 2; train_loss: 0.73; val_loss: 1.02; train_acc: 0.52; train_f1: 0.55, val_acc: 0.55; val_f1: 0.663
Epoch 3; train_loss: 0.725; val_loss: 1.085; train_acc: 0.532; train_f1: 0.568, val_acc: 0.6; val_f1: 0.717
Epoch 4; train_loss: 0.718; val_loss: 1.009; train_acc: 0.545; train_f1: 0.583, val_acc: 0.55; val_f1: 0.655
Epoch 5; train_loss: 0.704; val_loss: 0.925; train_acc: 0.569; train_f1: 0.593, val_acc: 0.608; val_f1: 0.718
Epoch 6; train_loss: 0.705; val_loss: 1.03; train_acc: 0.544; train_f1: 0.576, val_acc: 0.583; val_f1: 0.714
Epoch 7; train_loss: 0.686; val_loss: 0.849; train_acc: 0.591; train_f1: 0.623, val_acc: 0.608; val_f1: 0.717
Epoch 8; train_loss: 0.691; val_loss: 1.031; train_acc: 0.564; train_f1: 0.596, val_acc: 0.55; val_f1: 0.651
Epoch 9; train_loss: 0.

Epoch 75; train_loss: 0.474; val_loss: 0.735; train_acc: 0.785; train_f1: 0.786, val_acc: 0.667; val_f1: 0.674
Epoch 76; train_loss: 0.458; val_loss: 0.753; train_acc: 0.799; train_f1: 0.801, val_acc: 0.7; val_f1: 0.669
Epoch 77; train_loss: 0.451; val_loss: 0.71; train_acc: 0.786; train_f1: 0.791, val_acc: 0.692; val_f1: 0.679
Epoch 78; train_loss: 0.456; val_loss: 0.826; train_acc: 0.798; train_f1: 0.801, val_acc: 0.625; val_f1: 0.637
Epoch 79; train_loss: 0.454; val_loss: 0.735; train_acc: 0.784; train_f1: 0.79, val_acc: 0.683; val_f1: 0.642
Epoch 80; train_loss: 0.448; val_loss: 0.728; train_acc: 0.803; train_f1: 0.806, val_acc: 0.667; val_f1: 0.696
Epoch 81; train_loss: 0.446; val_loss: 0.717; train_acc: 0.799; train_f1: 0.809, val_acc: 0.683; val_f1: 0.689
Epoch 82; train_loss: 0.458; val_loss: 0.777; train_acc: 0.783; train_f1: 0.786, val_acc: 0.633; val_f1: 0.643
Epoch 83; train_loss: 0.445; val_loss: 0.689; train_acc: 0.796; train_f1: 0.801, val_acc: 0.7; val_f1: 0.684
Epoch 8

Epoch 149; train_loss: 0.401; val_loss: 0.814; train_acc: 0.824; train_f1: 0.825, val_acc: 0.608; val_f1: 0.6
Epoch 150; train_loss: 0.401; val_loss: 0.8; train_acc: 0.828; train_f1: 0.831, val_acc: 0.7; val_f1: 0.714
Epoch 151; train_loss: 0.396; val_loss: 0.799; train_acc: 0.824; train_f1: 0.829, val_acc: 0.692; val_f1: 0.74
Epoch 152; train_loss: 0.383; val_loss: 1.043; train_acc: 0.828; train_f1: 0.831, val_acc: 0.617; val_f1: 0.654
Epoch 153; train_loss: 0.39; val_loss: 0.905; train_acc: 0.829; train_f1: 0.834, val_acc: 0.642; val_f1: 0.669
Epoch 154; train_loss: 0.386; val_loss: 0.8; train_acc: 0.838; train_f1: 0.84, val_acc: 0.675; val_f1: 0.675
Epoch 155; train_loss: 0.405; val_loss: 0.863; train_acc: 0.813; train_f1: 0.815, val_acc: 0.675; val_f1: 0.644
Epoch 156; train_loss: 0.402; val_loss: 0.807; train_acc: 0.828; train_f1: 0.834, val_acc: 0.692; val_f1: 0.654
Epoch 157; train_loss: 0.401; val_loss: 0.807; train_acc: 0.824; train_f1: 0.821, val_acc: 0.7; val_f1: 0.63
Epoch 

## Running on Test dataset

In [9]:
preds = []
gt = []

for item in test_dataloader:
    with torch.no_grad():
        x, y = item[:, :-1], item[:, -1].reshape(-1, 1)
        y = y.double().to(device)
        x = x.float().to(device)
        y_tilde = model(x)
        y_tilde = y_tilde.double().to(device)
        y_pred = np.round(y_tilde.to('cpu').detach().numpy(), decimals = 0)
        y = np.round(y.to('cpu').detach().numpy(), decimals = 0)
        y = list(y)
        y_pred = list(y_pred)
        gt.extend(y)
        preds.extend(y_pred)
        

In [10]:
acc = accuracy_score(gt, preds)
prec = precision_score(gt, preds)
recall = recall_score(gt, preds)
f1 = f1_score(gt, preds)

##### ACC - 67 gan



In [11]:
acc

0.703125

In [12]:
print(len(preds))

128


In [13]:
print(len(test_data))

128
